# **ANALYSIS OF CHENNAI**

## **Introduction**

In this data analysis exercise, we are going to analyse the city of Chennai, Tamil nadu, India for a new tourist to know about the city. Chennai is located in the southern part of India and has the second largest beach in the world. It has many places to explore and is one of the important tourist destinations of the country. Hence, analysing the data of Chennai using foursquare api will be highly helpful for new people who are migrating to Chennai and also to the tourists.

### **Importing libraries**

In [1]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
!pip install geopy
from geopy.geocoders import Nominatim 
print ("done")


done


In [30]:
import numpy as np 
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [2]:
!pip install folium 
import folium

     |████████████████████████████████| 102kB 11.7MB/s ta 0:00:01


### **Getting coordinates of Chennai and displaying the map for reference using folium**

In [3]:
address = 'Chennai'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai are 13.0801721, 80.2838331.


In [4]:
import folium

chennai_map = folium.Map(location = [latitude, longitude], zoom_start = 12)
folium.Marker([latitude, longitude]).add_to(chennai_map)
chennai_map

## **Data**

Data for the venues in Chennai are taken from foursquare api. Foursquare consists of the details of places, their importance, latitudes, longitudes and which category they belong to. Hence, this api would be helpful to analyse the picture of Chennai. Data is collected using client id and secret created from foursquare website and used for the required analysis.

### **Obtaining data from foursquare api**

In [5]:
CLIENT_ID = '1T2KE0DYW1LHHFUYNAVZV5EKTBX2GZBJZDV3YLIAOQL4V2K2' # your Foursquare ID
CLIENT_SECRET = 'DE0YE2NSTQ2OCBZI3TIR4KCBCR3XYKOJLR03CC0ECXNXVHPE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1T2KE0DYW1LHHFUYNAVZV5EKTBX2GZBJZDV3YLIAOQL4V2K2
CLIENT_SECRET:DE0YE2NSTQ2OCBZI3TIR4KCBCR3XYKOJLR03CC0ECXNXVHPE


In [6]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1T2KE0DYW1LHHFUYNAVZV5EKTBX2GZBJZDV3YLIAOQL4V2K2&client_secret=DE0YE2NSTQ2OCBZI3TIR4KCBCR3XYKOJLR03CC0ECXNXVHPE&v=20180605&ll=13.0801721,80.2838331&radius=5000&limit=100'

## **Methodology**

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1479dc9bded205cfe3f47c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Chennai',
  'headerFullLocation': 'Chennai',
  'headerLocationGranularity': 'city',
  'totalResults': 77,
  'suggestedBounds': {'ne': {'lat': 13.125172145000045,
    'lng': 80.32994554231435},
   'sw': {'lat': 13.035172054999956, 'lng': 80.23772065768564}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4eeb7737aa1f66e35b2cff5d',
       'name': 'National Durbar Hotel',
       'location': {'address': '#3, Raja Muthiah Rd',
        'crossStreet': 'near Chennai Central. Eggmore',
        'lat': 13.081300779830098,
        'lng': 80.27060089302557,
        'labeledLatLngs':

In [8]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



### **Converting data in json format to a dataframe for further analysis**

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.shape

(77, 4)

In [18]:
nearby_venues.head()

,name,categories,lat,lng
0,National Durbar Hotel,Indian Restaurant,13.081301,80.270601
1,M A Chidambaram Stadium,Cricket Ground,13.062830,80.279239
2,Fort Museum,Museum,13.080618,80.287719
3,Bombay Lassi,Dessert Shop,13.066040,80.271222
4,Kakada Ramprasad,Indian Restaurant,13.090415,80.279111


### **Plotting the map of these venues**

In [20]:
chennai_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

for name, latitude, longitude in zip(nearby_venues['name'], nearby_venues['lat'], nearby_venues['lng']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [latitude, longitude],
        radius = 5,
        popup = label,
        color = 'green',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(chennai_map)  

chennai_map

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'categories', 
                  'lat', 
                  'lng', 
                ]
    
    return(nearby_venues)


### **Getting the names of the venues listed in map**

In [43]:
chennai_venues = getNearbyVenues(names=nearby_venues['name'],
                                   latitudes=nearby_venues['lat'],
                                   longitudes=nearby_venues['lng']
                                  )

National Durbar Hotel
M A Chidambaram Stadium
Fort Museum
Bombay Lassi
Kakada Ramprasad
Escape Cinemas
Mathura (woodland group)
Links
Ratna cafe
Blend - The High Energy Bar
Burma Bazaar
Sathyam Cinemas
Express Avenue
Seena bhai tiffen centre
Fruit Shop On Greams Road
Marina Beach | மெரினா கடற்கரை (Marina Beach)
Serene Sathyam
Krispy Kreme Doughnuts
Shree Mithai
Nair Mess
Blur
gabbana.life
Nicky's Cafe & Fine Pastries
Vivanta by Taj - Connemara
Streak
Marks & Spencers
Charminar Briyani And Tandoori Center
Kabab Corner
Sandy's Chocolate Laboratory
Alliance Francaise
Kasi Vinayaga Mess
French Loaf
Amadora Gourmet Ice Cream
Tuscana Pizzeria
Subway
The Marina
Hotel Welcome
Cream & Fudge
Light House
Z The Tapas Bar & Restaurant
Subway
The Brew Room
Bismi Biryani
Nasi And Mee
Dynasty
Little Italy Ristorante
Taj Coromandel
Higginbothams
AL - NAJEEB Nawab Of Kabab
Häagen-Dazs
Nando's
Hotel Crescent
Fu Silli Reasons
Hotel Saravana Bhavan
Titanic Fast Food
Bombaysthan
Gandhi Beach
Kannagi Statue


ValueError: Length mismatch: Expected axis has 7 elements, new values have 4 elements

In [44]:
nearby_venues.groupby('name').count()

,categories,lat,lng
name,,,
AL - NAJEEB Nawab Of Kabab,1,1,1
Adyar Ananda Bhavan,1,1,1
Alliance Francaise,1,1,1
Amadora Gourmet Ice Cream,1,1,1
Baskin-Robbins,1,1,1
Bismi Biryani,1,1,1
Blend - The High Energy Bar,1,1,1
Blur,1,1,1
Bombay Halwa House,1,1,1


### **One hot encoding**

In [28]:
ch_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

ch_onehot['name'] = nearby_venues['name'] 

fixed_columns = [ch_onehot.columns[-1]] + list(ch_onehot.columns[:-1])
ch_onehot = ch_onehot[fixed_columns]

ch_onehot.head()

,name,African Restaurant,Asian Restaurant,Bar,Beach,Bookstore,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,...,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Stadium,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Store
0,National Durbar Hotel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M A Chidambaram Stadium,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Fort Museum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bombay Lassi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Kakada Ramprasad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
ch_grouped = ch_onehot.groupby('name').mean().reset_index()
ch_grouped

,name,African Restaurant,Asian Restaurant,Bar,Beach,Bookstore,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,...,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Stadium,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Store
0,AL - NAJEEB Nawab Of Kabab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adyar Ananda Bhavan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alliance Francaise,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Amadora Gourmet Ice Cream,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Baskin-Robbins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Bismi Biryani,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Blend - The High Energy Bar,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Blur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Bombay Halwa House,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Bombay Lassi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### **Sorting the venues for further analysis**

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['name'] = ch_grouped['name']

for ind in np.arange(ch_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(ch_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AL - NAJEEB Nawab Of Kabab,Food,Video Store,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Fast Food Restaurant,Donut Shop,Dessert Shop
1,Adyar Ananda Bhavan,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
2,Alliance Francaise,Theater,Video Store,Cricket Ground,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
3,Amadora Gourmet Ice Cream,Ice Cream Shop,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
4,Baskin-Robbins,Ice Cream Shop,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop


### **CLUSTERING**

In [56]:
kclusters = 5

ch_clustering = ch_grouped.drop('name', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ch_clustering)

kmeans.labels_[0:10] 

array([0, 3, 0, 2, 2, 3, 0, 0, 3, 0], dtype=int32)

In [57]:
ch_merged = results

ch_merged = nearby_venues.join(venues_sorted.set_index('name'), on='name')

ch_merged.head() 

,name,categories,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,National Durbar Hotel,Indian Restaurant,13.081301,80.270601,3,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
1,M A Chidambaram Stadium,Cricket Ground,13.062830,80.279239,0,Cricket Ground,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
2,Fort Museum,Museum,13.080618,80.287719,0,Museum,Ice Cream Shop,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop,Department Store
3,Bombay Lassi,Dessert Shop,13.066040,80.271222,0,Dessert Shop,Video Store,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop
4,Kakada Ramprasad,Indian Restaurant,13.090415,80.279111,3,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop


In [58]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(ch_merged['lat'], ch_merged['lng'], ch_merged['name'], ch_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## **Results**

In [59]:
ch_merged.loc[ch_merged['Cluster Labels'] == 0, ch_merged.columns[[1] + list(range(5, ch_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Cricket Ground,Cricket Ground,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
2,Museum,Museum,Ice Cream Shop,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop,Department Store
3,Dessert Shop,Dessert Shop,Video Store,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop
5,Multiplex,Multiplex,Video Store,Ice Cream Shop,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
7,Snack Place,Snack Place,Video Store,Cricket Ground,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
9,Bar,Bar,Video Store,Department Store,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop
10,Video Store,Video Store,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
11,Movie Theater,Movie Theater,Video Store,Ice Cream Shop,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
12,Shopping Mall,Shopping Mall,Video Store,Cricket Ground,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
13,Restaurant,Restaurant,Video Store,Cricket Ground,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop


In [60]:
ch_merged.loc[ch_merged['Cluster Labels'] == 1, ch_merged.columns[[1] + list(range(5, ch_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Italian Restaurant,Italian Restaurant,Ice Cream Shop,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
45,Italian Restaurant,Italian Restaurant,Ice Cream Shop,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
52,Italian Restaurant,Italian Restaurant,Ice Cream Shop,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop


In [61]:
ch_merged.loc[ch_merged['Cluster Labels'] == 2, ch_merged.columns[[1] + list(range(5, ch_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Ice Cream Shop,Ice Cream Shop,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
37,Ice Cream Shop,Ice Cream Shop,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
49,Ice Cream Shop,Ice Cream Shop,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
72,Ice Cream Shop,Ice Cream Shop,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop


In [62]:
ch_merged.loc[ch_merged['Cluster Labels'] == 3, ch_merged.columns[[1] + list(range(5, ch_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
4,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
6,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
8,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
18,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
19,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
30,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
42,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
53,Vegetarian / Vegan Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop
58,Indian Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Fast Food Restaurant,Donut Shop,Dessert Shop


In [63]:
ch_merged.loc[ch_merged['Cluster Labels'] == 4, ch_merged.columns[[1] + list(range(5, ch_merged.shape[1]))]]

,categories,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Fast Food Restaurant,Fast Food Restaurant,Video Store,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Donut Shop,Dessert Shop
61,Fast Food Restaurant,Fast Food Restaurant,Video Store,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Donut Shop,Dessert Shop
69,Fast Food Restaurant,Fast Food Restaurant,Video Store,Vegetarian / Vegan Restaurant,Hyderabadi Restaurant,Hotel,Gaming Cafe,Frozen Yogurt Shop,Food,Donut Shop,Dessert Shop


## **Conclusion**

From the above analysis, it is seen that there is more number of vegetarian restaurants in and around Chennai. It is highly helpful for tourists who are vegan or vegetarians visiting Chennai.
There is also a cricket ground and multiplexes which are the places for entertainment to the tourists and also to the local people.
There are high number of ice cream parlours which indicates Chennai 